<a href="https://colab.research.google.com/github/VishanOberoi/Torch/blob/main/Fully_Connected_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data  import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Create a Fully Connected Network


In [ ]:


class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self,x):
    x  = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x



# Set device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 1
device

device(type='cuda')

#Load Data

In [ ]:
train_dataset = datasets.MNIST(root = 'dataset/', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size , shuffle = True)

test_dataset =  datasets.MNIST(root = 'dataset/', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size , shuffle = True)


100%|██████████| 9912422/9912422 [00:00<00:00, 100357518.28it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 68092014.52it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 25206717.27it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4410865.66it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



# Initialize the network



In [ ]:
model = NN(input_size = input_size, num_classes = num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)


#Train the network


In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device = device)
    targets = targets.to(device = device)
    data = data.reshape(data.shape[0], -1) #flatten to a single dim

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad() #set all gradients to zero for each new batch
    loss.backward()

    #Adam step
    optimizer.step() #Update weights









In [ ]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking acc on training data")
  else:
    print("Checking acc on testing data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad(): #Dont need to compute the gradients
    for x,y in loader:
      x = x.to(device = device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x) #There are 10 digits, give the max prob
      scores.max(1) #Shape is 64x10, We want vals for 10 part
      _, preds = scores.max(1)
      num_correct +=  (preds == y).sum()
      num_samples += preds.size(0)
    #Convert the tensors to floats
    print(f"Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100}")

  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking acc on training data
Got 55659/ 60000 with accuracy 92.765
Checking acc on testing data
Got 9280/ 10000 with accuracy 92.80000000000001


##These are the results after training one epoch on a very small network. Better on testing set.